In [16]:
import cv2;
import os;
import shutil;
import numpy as np;

In [17]:
from google.colab import drive;
drive.mount('/content/drive');

# path of dataset(in toys_in_the_room foler)
fpath = "/content/drive/MyDrive/Deep_Learning_group_six_upload";

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
def main() :
    
    # processed image size
    crop_size = (400, 400); # (w, h)
    
    # load data
    file_name, file_angle, ans_type_numpy = load_dataset(fpath, 1);
    print("num of image = {}\n".format(len(file_name)));
    
    # preprocess (find absolute coordinates (w, h) and crop image)
    abs_list = build_training_set(fpath, file_name, crop_size, 0);
    print(abs_list); # where the robot needs to move
    
    return(0);

In [19]:
# 讀取資料集

def load_dataset(path, Debug_mode) :
    
    # 讀取資料集中各圖片中的檔名以及其對應的夾取角度答案
    toys_numpy = np.loadtxt(path + "/img_grabbing_angle.csv", dtype = str, delimiter = ',', skiprows = 1);
    file_name = [i +".png" for i in toys_numpy[:,0]];
    file_angle = [int(i) for i in toys_numpy[:,1]];
    
    # 讀取此資料集中夾取角度的所有可能答案
    ans_type_numpy = np.loadtxt(path + "/angle_type.txt", dtype = int);

    if (Debug_mode) :
        print("filename {} :\n {}".format(type(file_name[0]), file_name));
        print("file_angle {} :\n {}".format(type(file_angle[0]), file_angle));
        print("ans_type {} :\n {}".format(type(ans_type_numpy[0]), ans_type_numpy));
    
    return(file_name, file_angle, ans_type_numpy);

In [20]:
# 對資料集進行預處理

def build_training_set(path, file_name, crop_size, Debug_mode) :
    
    
    save_path = path + "/preprocessed_images/";
    # delete old file in fold (toys_in_the_room)
    shutil.rmtree(save_path);
    os.mkdir(save_path);
    
    abs_list = [];
    
    for i in range (0, len(file_name)) :
        
        img = cv2.imread(path + "/toys_in_the_room/"  + file_name[i]);
        
        # find target
        cnt_max = findMarker(img, 0);
        rect = cv2.minAreaRect(cnt_max);
        box = np.int0(cv2.boxPoints(rect));
        if (Debug_mode) :
            cv2.drawContours(img, [box], 0, (0,0,255), 2);
        
        # find absolute coordinates (w, h)
        max_h = box[0][1];
        for j in range (0, len(box)) :
            if (max_h < box[j][1]) :
                max_h = box[j][1];
        
        abs_position = (int(rect[0][0]), max_h);
        abs_list.append(abs_position);
        
        # crop image
        ULHC_w = max(int(rect[0][0] - crop_size[0]*0.5), 0); # upper left hand corner (w, h)
        ULHC_h = max(int(rect[0][1] - crop_size[1]*0.5), 0);
        LRHC_w = min(int(rect[0][0] + crop_size[0]*0.5), img.shape[1]); # lower right hand corner (w, h)
        LRHC_h = min(int(rect[0][1] + crop_size[1]*0.5), img.shape[1]);
        
        crop_img = img[ULHC_h:LRHC_h, ULHC_w:LRHC_w]; # [h, w]

        #if (Debug_mode) : # 將處理過的資料集存下，來檢查此預處理是否能確實的擷取到物品所在的範圍
        cv2.imwrite(save_path + file_name[i], crop_img);

        if (0) :
          print("filename : ", file_name[i]);
          print(ULHC_w, ULHC_h, LRHC_w, LRHC_h);
          cv2.imshow("crop_img", crop_img);
          cv2.waitKey();
        
    return (abs_list);

def findMarker(image, Debug_mode) :
    
    kernel_size = 13;
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY);
    blur = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0);
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2);
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE);
    cnt_max = max(contours, key = cv2.contourArea);
    
    if (Debug_mode) :
        cv2.imshow("gray", gray);
        cv2.imshow("blur", blur);
        cv2.imshow("thresh", thresh);
        print("num_contour = ", len(contours));
    
    return(cnt_max);

In [21]:
main();

filename <class 'str'> :
 ['doll7_135.png', 'doll7_0.png', 'doll7_90.png', 'doll7_45.png', 'logoman1_0.png', 'logoman1_90.png', 'logoman1_135.png', 'logoman1_45.png', 'logoman2_0.png', 'logoman2_90.png', 'logoman2_45.png', 'logoman2_135.png', 'logoman3_90.png', 'logoman3_135.png', 'logoman3_0.png', 'logoman3_45.png', 'doll12_0.png', 'doll12_135.png', 'doll12_90.png', 'doll12_45.png', 'doll11_0.png', 'doll11_90.png', 'doll11_45.png', 'doll11_135.png', 'doll9_90.png', 'doll9_135.png', 'doll9_0.png', 'doll9_45.png', 'doll6_45.png', 'doll6_90.png', 'doll6_0.png', 'doll6_135.png', 'car10_0.png', 'car10_90.png', 'car10_135.png', 'car10_45.png', 'doll4_45.png', 'doll4_0.png', 'doll4_90.png', 'doll4_135.png', 'doll5_90.png', 'doll5_45.png', 'doll5_135.png', 'doll5_0.png', 'car3_0.png', 'car3_45.png', 'car3_135.png', 'car3_90.png', 'car2_90.png', 'car2_0.png', 'car2_135.png', 'car2_45.png', 'car4_0.png', 'car4_135.png', 'car4_90.png', 'car4_45.png', 'doll10_135.png', 'doll10_45.png', 'doll10_0.